In [2]:
import pandas as pd
import numpy as np
from bson import ObjectId

import keyring
import mysql.connector

import arcgis
from arcgis.gis import GIS
import datetime as dt
import getpass
from IPython.display import HTML

from arcgis.geocoding import geocode
from arcgis.features import Feature, FeatureSet, FeatureLayerCollection, use_proximity, FeatureLayer
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.network.analysis import generate_origin_destination_cost_matrix

In [3]:
username = "Glodanale"
password = keyring.get_password("arcgis", "Glodanale")
gis = GIS("https://www.arcgis.com", "Glodanale", password)

In [29]:
def pullEIDataFromMySQL():
    connection = mysql.connector.connect(
            host = "localhost",
            port = 3306,
            user = 'root',
            database = "expandinghorizons",
            password = "dB79@dG2024!"
    )
    
    cursor = connection.cursor()
    
    query = "SELECT * FROM ei"
    cursor.execute(query)
    
    rows = cursor.fetchall()
    
    df = pd.DataFrame(rows, columns=['_id', 'FirstName', 'LastName', 'ServiceHours', 'Address1', 'City', 'State', 'Zipcode', 'County', 'Longitude', 'Latitude'])
    df["Role"] = "ei"
    
    cursor.close()
    connection.close()
    
    return df

In [30]:
def pullChildDataFromMySQL():
    connection = mysql.connector.connect(
            host = "localhost",
            port = 3306,
            user = 'root',
            database = "expandinghorizons",
            password = "dB79@dG2024!"
    )
    
    cursor = connection.cursor()
    
    query = "SELECT * FROM child"
    cursor.execute(query)
    
    rows = cursor.fetchall()
    
    df = pd.DataFrame(rows, columns=['_id', 'FirstName', 'LastName', 'ServiceHours', 'Address1', 'City', 'State', 'Zipcode', 'County', 'Longitude', 'Latitude'])
    df["Role"] = "child"
    
    cursor.close()
    connection.close()
    
    return df

In [31]:
dataEI = pullEIDataFromMySQL()
dataChild = pullChildDataFromMySQL()

In [32]:
allData = pd.concat([dataEI, dataChild], ignore_index=True)
print(allData)

                          _id FirstName    LastName  ServiceHours  \
0    65d75a2ce09e67e025e8cd30      Mary  Richardson           0.0   
1    65d75bb0e09e67e025e8cd31   Kirstin      Tipton           1.0   
2    65d75c59e09e67e025e8cd32     Kassi      Knight           0.0   
3    65d75c87e09e67e025e8cd33    Rachel      Taylor           0.0   
4    65d75cb9e09e67e025e8cd34   Brannan     Vanover           0.0   
..                        ...       ...         ...           ...   
204  65e6038938f826f29267202f     Judah     Overbay           1.0   
205  65e603f238f826f292672030    Bianca       Pogue           1.0   
206  65e6044938f826f292672031   Emorett     Woodall           1.0   
207  65e6049b38f826f292672032      Remi       Woody           0.5   
208  65ef076606e0a866af56d202  Madilynn      Phipps           1.0   

                      Address1           City State Zipcode      County  \
0     904 West Hillcrest Drive   Johnson City    TN   37604  Washington   
1        151 Valley V

In [33]:
def combine_address(row):
    return f"{row['Address1']}, {row['City']}, {row['State']} {row['Zipcode']}"

allData["Full_Address"] = allData.apply(combine_address, axis=1)
#print(allData)

featureSetDF = allData.copy()
featureSetDF = allData.drop(columns=["ServiceHours", "Address1", "City", "State", "Zipcode", "County"])
featureSetDF = featureSetDF[["_id","FirstName", "LastName", "Full_Address", "Longitude", "Latitude", "Role"]]
print(featureSetDF)

                          _id FirstName    LastName  \
0    65d75a2ce09e67e025e8cd30      Mary  Richardson   
1    65d75bb0e09e67e025e8cd31   Kirstin      Tipton   
2    65d75c59e09e67e025e8cd32     Kassi      Knight   
3    65d75c87e09e67e025e8cd33    Rachel      Taylor   
4    65d75cb9e09e67e025e8cd34   Brannan     Vanover   
..                        ...       ...         ...   
204  65e6038938f826f29267202f     Judah     Overbay   
205  65e603f238f826f292672030    Bianca       Pogue   
206  65e6044938f826f292672031   Emorett     Woodall   
207  65e6049b38f826f292672032      Remi       Woody   
208  65ef076606e0a866af56d202  Madilynn      Phipps   

                                         Full_Address  Longitude   Latitude  \
0    904 West Hillcrest Drive, Johnson City, TN 37604 -82.372108  36.319485   
1           151 Valley View Drive, Abingdon, VA 24210 -81.977836  36.722124   
2        7040 Blue Springs Parkway, Mosheim, TN 37818 -82.944595  36.197052   
3            502 Nunley

In [34]:
specific_ids = ['65e5fe7438f826f292672025', '65e5d57838f826f292671ffd']
filtered_df = featureSetDF.loc[featureSetDF['_id'].isin(specific_ids)]

print(filtered_df)
print("----------------------------------------------------------------------------------------------")
print(featureSetDF)

                          _id FirstName LastName  \
155  65e5d57838f826f292671ffd    Ledger    Green   
194  65e5fe7438f826f292672025    Colton  Hanczyk   

                                    Full_Address  Longitude   Latitude   Role  
155  6345 Wildlife Circle, Piney Flats, TN 37686 -82.374298  36.409046  child  
194          518 Ford Creek Road, Gray, TN 37615 -82.504990  36.419319  child  
----------------------------------------------------------------------------------------------
                          _id FirstName    LastName  \
0    65d75a2ce09e67e025e8cd30      Mary  Richardson   
1    65d75bb0e09e67e025e8cd31   Kirstin      Tipton   
2    65d75c59e09e67e025e8cd32     Kassi      Knight   
3    65d75c87e09e67e025e8cd33    Rachel      Taylor   
4    65d75cb9e09e67e025e8cd34   Brannan     Vanover   
..                        ...       ...         ...   
204  65e6038938f826f29267202f     Judah     Overbay   
205  65e603f238f826f292672030    Bianca       Pogue   
206  65e60449

In [35]:
duplicates = featureSetDF[featureSetDF.duplicated(subset=["Longitude", "Latitude"], keep=False)]
print(duplicates)

                          _id    FirstName   LastName  \
40   65d77fdce09e67e025e8cd62        Dixie      Jones   
41   65d78068e09e67e025e8cd63       Gracie      Jones   
42   65d780cee09e67e025e8cd64     Benjamin   Keasling   
43   65d78173e09e67e025e8cd65        Henry   Keasling   
48   65d7842ee09e67e025e8cd6a       Hunter     Morton   
49   65d78493e09e67e025e8cd6b         Luke     Morton   
55   65d787f5e09e67e025e8cd71      Emersyn      Jones   
56   65d7886ae09e67e025e8cd72       Olivia      Jones   
65   65d8b679e09e67e025e8cd7b       Larson     Ketron   
66   65d8b6f8e09e67e025e8cd7c        Mason     Ketron   
67   65d8b770e09e67e025e8cd7d  Christopher      Kimes   
68   65d8b7d1e09e67e025e8cd7e       Corbin      Kimes   
69   65d8b845e09e67e025e8cd7f         Aria  McClellan   
70   65d8b8b7e09e67e025e8cd80       Aurora  McClellan   
72   65d8b9c0e09e67e025e8cd82      Charlie    Mullins   
76   65d8bc1ee09e67e025e8cd86       Armani        Arp   
80   65d8be2ae09e67e025e8cd8a  

In [36]:
uniqueFeatureSetDF = featureSetDF.copy()
uniqueFeatureSetDF = uniqueFeatureSetDF.drop_duplicates(subset=["Longitude", "Latitude"], keep="first")

uniqueFeatureSetDF = uniqueFeatureSetDF[~(uniqueFeatureSetDF['_id'] == "65e5d4d238f826f292671ffc")]
uniqueFeatureSetDF = uniqueFeatureSetDF[~(uniqueFeatureSetDF['_id'] == "65e5d4d238f826f292671ffc")]
uniqueFeatureSetDF = uniqueFeatureSetDF[~(uniqueFeatureSetDF['_id'] == "65e5d4d238f826f292671ffc")]
uniqueFeatureSetDF = uniqueFeatureSetDF[~(uniqueFeatureSetDF['_id'] == "65e5fe7438f826f292672025")]


uniqueFeatureSetDF = uniqueFeatureSetDF.reset_index(drop=True)
print(uniqueFeatureSetDF)

                          _id FirstName    LastName  \
0    65d75a2ce09e67e025e8cd30      Mary  Richardson   
1    65d75bb0e09e67e025e8cd31   Kirstin      Tipton   
2    65d75c59e09e67e025e8cd32     Kassi      Knight   
3    65d75c87e09e67e025e8cd33    Rachel      Taylor   
4    65d75cb9e09e67e025e8cd34   Brannan     Vanover   
..                        ...       ...         ...   
183  65e6038938f826f29267202f     Judah     Overbay   
184  65e603f238f826f292672030    Bianca       Pogue   
185  65e6044938f826f292672031   Emorett     Woodall   
186  65e6049b38f826f292672032      Remi       Woody   
187  65ef076606e0a866af56d202  Madilynn      Phipps   

                                         Full_Address  Longitude   Latitude  \
0    904 West Hillcrest Drive, Johnson City, TN 37604 -82.372108  36.319485   
1           151 Valley View Drive, Abingdon, VA 24210 -81.977836  36.722124   
2        7040 Blue Springs Parkway, Mosheim, TN 37818 -82.944595  36.197052   
3            502 Nunley

In [52]:
def buildFeatureSet(df):
    features = []

    for index, row in df.iterrows():
        attributes = {"Address": row['Full_Address'], "OBJECTID": row['_id'], "Role": row['Role']}
        geometry = {"x": row["Longitude"], 'y': row["Latitude"]}
        feature = {"attributes": attributes, "geometry": geometry}
        features.append(feature)
        
    location_json = {
        "features": features,
        "spatialReference": {"wkid": 4326, "lastestWkid": 4326},
        "geometryType": "esriGeometryPoint",
        "fields": [
            {"name": "OBJECTID", "type": "esriFieldTypeString", "alias": "OBJECTID", "length": "50"},
            {"name": "Address", "type": "esriFieldTypeString", "alias": "Address", "length": "100"},
            {"name": "Role", "type": "esriFieldTypeString", "alias": "Role", "length": "50"}
        ]
    }
        
    feature_set = FeatureSet.from_dict(location_json)
    return feature_set

In [38]:
def createFeatureLayer(feature_set, name):
    feature_layer_item = gis.content.import_data(feature_set, title=f"Feature Layer {name}")
    feature_layer_url = feature_layer_item.url
    #print("Feature Layer URL:", feature_layer_url)
    
    return feature_layer_item

In [39]:
def analyzeFeatureLayer(feature_layer_item):
    feature_layer = feature_layer_item.layers[0]
    features = feature_layer.query()

    for feature in features:
        print(feature)
        print("\n")

In [40]:
def process(df, name):
    feature_set = buildFeatureSet(df)
    feature_layer_item = createFeatureLayer(feature_set, name)
    analyzeFeatureLayer(feature_layer_item)
    
    return feature_layer_item

In [41]:
def buildMatrix(origin, destination, name):
    items = gis.content.search(query="OD Cost Matrix", item_type="Feature Service")
    for item in items:
        item.delete()

    result = use_proximity.connect_origins_to_destinations(origins_layer=origin,
                                                            destinations_layer=destination,
                                                            #origins_layer_route_id_field= "Status",
                                                            #destinations_layer_route_id_field="Status",
                                                            output_name=f"OD Cost Matrix {name}",
                                                            gis=gis)

    print("I completed the function")

    od_cost_matrix_sublayer = FeatureLayer.fromitem(result, layer_id=0)
    od_df = od_cost_matrix_sublayer.query(where='1=1', as_df=True)
    myODMatrix = od_df[['From_OBJECTID','From_Address','From_Role','To_OBJECTID','To_Address','To_Role','Total_Miles','Total_Minutes']]
    print(myODMatrix)

    return myODMatrix

In [42]:
def child_child_entry(layerID, loopID, cursor, miles=0.0, minutes=0.0):
    query = ("SELECT * FROM locationdata "
             f"WHERE child_id_1 = '{layerID}' and child_id_2 = '{loopID}' "
             f"OR child_id_1 = '{loopID}' and child_id_2 = '{layerID}'" )     

    cursor.execute(query)
    rows = cursor.fetchall()

    if rows:
        updateQuery = ("UPDATE locationdata " 
                       f"SET driving_distance = {miles}, driving_time = {minutes} " 
                       f"WHERE child_id_1 = '{layerID}' and child_id_2 = '{loopID}' "
                       f"OR child_id_1 = '{loopID}' and child_id_2 = '{layerID}'" )

        cursor.execute(updateQuery)
        print(f"{layerID} to {loopID} is updated in the MySQL database")
    else:
        insertQuery = ("INSERT INTO locationdata(child_id_1, child_id_2, ei_id_1, ei_id_2, assigned, driving_distance, driving_time) "
                       "VALUES (%s, %s, %s, %s, %s, %s, %s) ")
        cursor.execute(insertQuery, (
            layerID, loopID, None, None, 0, miles, minutes))
        print(f"{layerID} to {loopID} is inserted in the MySQL database")

In [43]:
def child_ei_entry(layerID, loopID, cursor, miles=0.0, minutes=0.0):
    query = ("SELECT * FROM locationdata "
             f"WHERE child_id_1 = '{layerID}' and ei_id_1 = '{loopID}'")

    cursor.execute(query)
    rows = cursor.fetchall()

    if rows:
        updateQuery = ("UPDATE locationdata " 
                       f"SET driving_distance = {miles}, driving_time = {minutes} " 
                       f"WHERE child_id_1 = '{layerID}' and ei_id_1 = '{loopID}'")

        cursor.execute(updateQuery)
        print(f"{layerID} to {loopID} is updated in the MySQL database")
    else:
        insertQuery = ("INSERT INTO locationdata(child_id_1, child_id_2, ei_id_1, ei_id_2, assigned, driving_distance, driving_time) "
                       "VALUES (%s, %s, %s, %s, %s, %s, %s) ")
        cursor.execute(insertQuery, (
            layerID, None, loopID, None, 0, miles, minutes))
        print(f"{layerID} to {loopID} is inserted in the MySQL database")

In [44]:
def ei_child_entry(layerID, loopID, cursor, miles=0.0, minutes=0.0):
    query = ("SELECT * FROM locationdata "
             f"WHERE ei_id_1 = '{layerID}' and child_id_1 = '{loopID}'")
    print(query)

    cursor.execute(query)
    rows = cursor.fetchall()

    if rows:
        updateQuery = ("UPDATE locationdata " 
                       f"SET driving_distance = {miles}, driving_time = {minutes} " 
                       f"WHERE ei_id_1 = '{layerID}' and child_id_1 = '{loopID}'")

        cursor.execute(updateQuery)
        print(f"{layerID} to {loopID} is updated in the MySQL database")
    else:
        insertQuery = ("INSERT INTO locationdata(child_id_1, child_id_2, ei_id_1, ei_id_2, assigned, driving_distance, driving_time) "
                       "VALUES (%s, %s, %s, %s, %s, %s, %s) ")
        cursor.execute(insertQuery, (
            loopID, None, layerID, None, 0, miles, minutes))
        print(f"{layerID} to {loopID} is inserted in the MySQL database")

In [45]:
def ei_ei_entry(layerID, loopID, cursor, miles=0.0, minutes=0.0):
    query = ("SELECT * FROM locationdata "
             f"WHERE ei_id_1 = '{layerID}' and ei_id_2 = '{loopID}' "
             f"OR ei_id_1 = '{loopID}' and ei_id_2 = '{layerID}'")

    cursor.execute(query)
    rows = cursor.fetchall()

    if rows:
        updateQuery = ("UPDATE locationdata " 
                       f"SET driving_distance = {miles}, driving_time = {minutes} " 
                       f"WHERE ei_id_1 = '{layerID}' and ei_id_2 = '{loopID}' "
                       f"OR ei_id_1 = '{loopID}' and ei_id_2 = '{layerID}'")

        cursor.execute(updateQuery)
        print(f"{layerID} to {loopID} is updated in the MySQL database")

    else:
        insertQuery = ("INSERT INTO locationdata(child_id_1, child_id_2, ei_id_1, ei_id_2, assigned, driving_distance, driving_time) "
                       "VALUES (%s, %s, %s, %s, %s, %s, %s) ")
        cursor.execute(insertQuery, (
            None, None, layerID, loopID, 0, miles, minutes))
        print(f"{layerID} to {loopID} is inserted in the MySQL database")

In [46]:
def pushLocationDataToMySQL(locationDF, duplicates):
    if locationDF.empty:
        print("Error: locationDF is empty")
    else:
        connection = mysql.connector.connect(
            host = "localhost",
            port = 3306,
            user = 'root',
            database = "expandinghorizons",
            password = "dB79@dG2024!"
        )
    
        cursor = connection.cursor()
    
        if cursor:
            print(f"\n\nDatabase connected\n\n")
        else:
            print("not connected")
    
        for index, row in locationDF.iterrows():
            print()
            if row.isnull().any():
                print(f"{row['From_OBJECTID']} to {row['To_OBJECTID']} contains null values")
            else:
                fromRole = row["From_Role"]
                toRole = row["To_Role"]
                miles = row['Total_Miles']
                minutes = row['Total_Minutes']
                fromID = row["From_OBJECTID"]
                toID = row["To_OBJECTID"]
                
                print(f"fromRole: {fromRole}   toRole:{toRole}")
                
                #make sure duplicates are entered in locationdata with 0 distance and 0 drive time
                if fromID in duplicates["_id"].values:
                    print("duplicate in fromID found")
                    from_filtered_ids = duplicates.loc[duplicates["Full_Address"] == row["From_Address"], "_id"].tolist()
                    from_filtered_roles = duplicates.loc[duplicates["Full_Address"] == row["From_Address"], "Role"].tolist()
                    for a in range(0, len(from_filtered_ids)):
                        loopID = from_filtered_ids[a]
                        loopRole = from_filtered_roles[a]
                        if fromID != loopID:
                            if fromRole == "child":
                                if loopRole == "child":
                                    child_child_entry(fromID, loopID, cursor)
                                elif loopRole == "ei":
                                    child_ei_entry(fromID, loopID, cursor)
                                else:
                                    print("Error with types")
                            elif fromRole == "ei":
                                if loopRole == "child":
                                    ei_child_entry(fromID, loopID, cursor)
                                elif loopRole == "ei":
                                    ei_ei_entry(fromID, loopID, cursor)
                                else:
                                    print("Error with types")
                            else:
                                print("Error finding role")
                        else:
                            print(f"IDs match: fromID={fromID}   loopID={loopID}")
                    #exit from_filtered_ids loop
                else:
                    from_filtered_ids = [fromID]
                    from_filtered_roles = [fromRole]
                         
                if toID in duplicates["_id"].values:
                    print("duplicate in toID found")
                    to_filtered_ids = duplicates.loc[duplicates["Full_Address"] == row["To_Address"], "_id"].tolist()
                    to_filtered_roles = duplicates.loc[duplicates["Full_Address"] == row["To_Address"], "Role"].tolist()
                    for d in range(0, len(to_filtered_ids)):
                        loopID = to_filtered_ids[d]
                        loopRole = to_filtered_roles[d]
                        if toID != loopID:
                            if toRole == "child":
                                if loopRole == "child":
                                    child_child_entry(toID, loopID, cursor)
                                elif loopRole == "ei":
                                    child_ei_entry(toID, loopID, cursor)
                                else:
                                    print("Error with types")
                            elif toRole == "ei":
                                if loopRole == "child":
                                    ei_child_entry(toID, loopID, cursor)
                                elif loopRole == "ei":
                                    ei_ei_entry(toID, loopID, cursor)
                                else:
                                    print("Error with types")
                            else:
                                print("Error finding role")
                        else:
                            print(f"IDs match: loopID={loopID}   toID={toID}")
                else:
                    to_filtered_ids = [toID]
                    to_filtered_roles = [toRole]
                #end of duplicate checking    
                
                #match all from_locations with all to_locations
                for i in range(0, len(from_filtered_ids)):
                    for j in range(0, len(to_filtered_ids)):
                        fromID = from_filtered_ids[i]
                        fromRole = from_filtered_roles[i]
                        toID = to_filtered_ids[j]
                        toRole = to_filtered_roles[j]
                    
                        if fromRole == "child":
                            if toRole == "child":
                                child_child_entry(fromID, toID, cursor, miles=miles, minutes=minutes)
                            elif toRole == "ei":
                                child_ei_entry(fromID, toID, cursor, miles=miles, minutes=minutes)
                            else:
                                print("Error recieving to_role")
                        elif fromRole == "ei":
                            if toRole == "child":
                                ei_child_entry(fromID, toID, cursor, miles=miles, minutes=minutes)                                    
                            elif toRole == "ei":
                                ei_ei_entry(fromID, toID, cursor, miles=miles, minutes=minutes)                                
                            else:
                                print("Error recieving to_role")
                        else:
                            print("Error recieving from_role")
                #end of all matches
 
        connection.commit()
        cursor.close()
        connection.close()
        
        print("---------------------------------------------------------------------------------------------------------------------------------")
        print("All data entered into database")

In [47]:
def buildData(df, uniqueFeatureSetDF, duplicates): 
    newUniqueFeatureSetDF = uniqueFeatureSetDF.copy()
    for index, row in df.iterrows():        
        row_df = pd.DataFrame(row).transpose()
        row_df.reset_index(drop=True, inplace=True)
        if len(row_df) != 1:
            print("Error in df length")
        else:
            id_value = row_df.loc[0, "_id"]
            newUniqueFeatureSetDF = newUniqueFeatureSetDF[newUniqueFeatureSetDF['_id'] != id_value]
            newUniqueFeatureSetDF.reset_index(drop=True, inplace=True)
            destination = process(newUniqueFeatureSetDF, 'destination')
            originLayer = process(row_df, id_value)
            originDestinationDF = buildMatrix(originLayer, destination, id_value)
            pushLocationDataToMySQL(originDestinationDF, duplicates)
            
            print("//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////")
            print("//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////")
            print("//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////")
    print("\n\n")
    print("Complete Function")
    
    return newUniqueFeatureSetDF

In [48]:
filtered_df = filtered_df[~(filtered_df['_id'] == "65e5d57838f826f292671ffd")]
filtered_df = filtered_df.reset_index(drop=True)

In [49]:
print(filtered_df)

                        _id FirstName LastName  \
0  65e5fe7438f826f292672025    Colton  Hanczyk   

                          Full_Address  Longitude   Latitude   Role  
0  518 Ford Creek Road, Gray, TN 37615  -82.50499  36.419319  child  


In [50]:
uniqueFeatureSetDF.reset_index(drop=True)
print(uniqueFeatureSetDF)

                          _id FirstName    LastName  \
0    65d75a2ce09e67e025e8cd30      Mary  Richardson   
1    65d75bb0e09e67e025e8cd31   Kirstin      Tipton   
2    65d75c59e09e67e025e8cd32     Kassi      Knight   
3    65d75c87e09e67e025e8cd33    Rachel      Taylor   
4    65d75cb9e09e67e025e8cd34   Brannan     Vanover   
..                        ...       ...         ...   
183  65e6038938f826f29267202f     Judah     Overbay   
184  65e603f238f826f292672030    Bianca       Pogue   
185  65e6044938f826f292672031   Emorett     Woodall   
186  65e6049b38f826f292672032      Remi       Woody   
187  65ef076606e0a866af56d202  Madilynn      Phipps   

                                         Full_Address  Longitude   Latitude  \
0    904 West Hillcrest Drive, Johnson City, TN 37604 -82.372108  36.319485   
1           151 Valley View Drive, Abingdon, VA 24210 -81.977836  36.722124   
2        7040 Blue Springs Parkway, Mosheim, TN 37818 -82.944595  36.197052   
3            502 Nunley

In [53]:
newFeatureSetDF = buildData(filtered_df, uniqueFeatureSetDF, duplicates)

{"geometry": {"x": -9169621.169276798, "y": 4344671.49332616, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 1, "Address": "904 West Hillcrest Drive, Johnson City, TN 37604", "OBJECTID": "65d75a2ce09e67e025e8cd30", "Role": "ei"}}


{"geometry": {"x": -9125730.959520198, "y": 4400444.975741492, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 2, "Address": "151 Valley View Drive, Abingdon, VA 24210", "OBJECTID": "65d75bb0e09e67e025e8cd31", "Role": "ei"}}


{"geometry": {"x": -9233350.116959408, "y": 4327769.409124248, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 3, "Address": "7040 Blue Springs Parkway, Mosheim, TN 37818", "OBJECTID": "65d75c59e09e67e025e8cd32", "Role": "ei"}}


{"geometry": {"x": -9174429.90765625, "y": 4342721.095604265, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 4, "Address": "502 Nunley Drive, Johnson City, TN 37604", "OBJECTID"

Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Avoid Unpaved Roads, Through Traffic Prohibited").
Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Through Traffic Prohibited").
{"messageCode": "AO_100138", "message": "Some origins were not connected to their paired destinations. Check the Status field in output unassigned origins and unassigned destinations layers for more information."}
{"cost": 0.935}


I completed the function


KeyError: "None of [Index(['From_OBJECTID', 'From_Address', 'From_Role', 'To_OBJECTID',\n       'To_Address', 'To_Role', 'Total_Miles', 'Total_Minutes'],\n      dtype='object')] are in the [columns]"